In [1]:
import csv
import os
import classes as vp
import plot
import mathHelper as mh

In [2]:
def loadSessions(file = 'sessions.csv', csv_data_dir = './csv_data'):
    #load session data form sessions.csv
    sessions = [] #list of all the sessions
    with open(file) as sessions_csv:
        sessionno = 0
        csv_reader = csv.reader(sessions_csv, delimiter=',')
        for l in csv_reader:
            sessions.append(vp.Session(sessionno, l[1], l[2], csv_data_dir + '/' + l[3]))
            sessionno += 1
    return sessions
    

In [3]:
#todo: need output
def loadFactions(file = 'factions.csv'):
    with open(file) as factions_csv:
    lineno = 0
    csv_reader = csv.reader(factions_csv, delimiter=',')
    for line in csv_reader:
        if lineno != 0:
            factions.append(line[1])
            memno.append(line[2])
        lineno += 1

IndentationError: expected an indented block (<ipython-input-3-4df89e1468f9>, line 4)

In [4]:
def parseMems(r, session):
    posIdMap = []
    line = next(r)
    start = 0
    end = 0
    for i, item in enumerate(line):
        if item == 'CouncillorId':
            start = i+1
            break
    
    next(r)
    
    #enumerate ID
    for i, item in enumerate(line[start:]):
        if item == '':
            end = i + start
            break
            
        posIdMap.append(item)
        if not item in members:
            members[item] = vp.Member(item)
        (members[item].ismem).append(session.idno)

    ''' 
    #enumerate bioid
    line = next(r)    
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].bioid = item
    '''
    
    #enumerate name
    line = next(r)
    for i,item in enumerate(line[start:end]):
        #print(i), print(item)
        members[posIdMap[i]].name = item
        
    #skip line (this line only shows members are part of national council)
    next(r)
    #enumerate faction membership
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].faction = item 
        
    #enumerate canton
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].canton = item
        
    #enumerate birthdata
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].birthdate = item

    

In [46]:
#parses all the sessions for the votes
def parseVotes(r, s):
    l = next(r)
    voters = []
    #votes = []
    voteno = 0
    start = 0
    while l[0] != 'VoteDate':
        l = next(r)
        start += 1
    
    #voters contains all the voters in a session
    endVotes = 0
    for i, item in enumerate(l[12:]):
        if item != 'Decision':
            voters.append(item)
        else:
            endVotes = i + 12
            break
    #print(len(voters))
    
    g = set()
    l = next(r)
    
    
    #need to handle:
    vote_none = '' #member was not yet or is no longer a national coucil member
    vote_yes = 'Ja'
    vote_no = 'Nein'
    vote_novote = ['Entschuldigt', 'Hat nicht teilgenommen']
    vote_abstain = 'Enthaltung'
    vote_pres = 'Der Präsident stimmt nicht'
    decision_set = {'yes', 'no'}
    
    
    while True:
        try:
            #store vote meta data
            v = vp.Vote(voteno)
            v.date = l[0]
            v.affairId = l[4]
            v.title = l[5]
            v.session = s
            #store how each member voted by adding the members bioid to the yes/no/abstain/noset set
            for i, item in enumerate(l[12:endVotes]):
                if item == vote_none:
                    pass
                elif item == vote_yes:
                    v.yes.add(voters[i])
                elif item == vote_no:
                    v.no.add(voters[i])
                elif item == vote_abstain:
                    v.abstain.add(voters[i])
                elif item in vote_novote:
                    v.novote.add(voters[i])
                elif item == vote_pres:
                    pass
                else:
                    print('err vote: ' + item)
            
            #store decision data as stored in file and compare with calculated decision value
            v.decision = l[endVotes].lower()
            
            if v.decision != v.getDecision() or (not v.decision in decision_set):
                v.decision = 'err: \"' + v.decision + '\"' 
            votes.append(v)
            voteno += 1
            l = next(r)
        except StopIteration:
            break

In [41]:
def parseFiles(sessions, toParse = {}): 
    #parse all the files
    #for testing: "filesToParse" can limit the parsed files (everything if empty)
    for s in sessions:
        if s.idno in toParse or len(toParse) == 0:
            with open(s.path) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                print('-', end='')
                #print(str(key) + '. reading ... ' + csv_file.name)
                #parseFile(csv_reader)
                #parseMems(csv_reader, s)
                parseVotes(csv_reader, s)
                csv_file.close

In [42]:
members = dict()
votes = []
factions = dict()
#factions = []
memno = []

In [43]:
sessions = loadSessions()
parseFiles(sessions, toParse={0})




-

In [44]:
'A'.lower()

'a'

In [45]:
for v in votes:
    print(v)

0 yes yes:114 no:42 abstain:0 novote:43
1 yes yes:88 no:77 abstain:1 novote:33
2 yes yes:157 no:0 abstain:0 novote:42
3 no yes:76 no:90 abstain:0 novote:33
4 yes yes:124 no:51 abstain:5 novote:19
5 yes yes:125 no:51 abstain:5 novote:18
6 yes yes:125 no:58 abstain:2 novote:14
7 yes yes:103 no:77 abstain:3 novote:16
8 yes yes:116 no:52 abstain:4 novote:27
9 yes yes:124 no:48 abstain:3 novote:24
10 no yes:74 no:100 abstain:2 novote:23
11 yes yes:129 no:48 abstain:0 novote:22
12 yes yes:95 no:90 abstain:1 novote:13
13 yes yes:94 no:86 abstain:3 novote:16
14 yes yes:131 no:43 abstain:9 novote:16
15 yes yes:99 no:59 abstain:0 novote:41
16 no yes:83 no:88 abstain:1 novote:27
17 yes yes:105 no:73 abstain:0 novote:21
18 yes yes:93 no:92 abstain:1 novote:14
19 yes yes:115 no:72 abstain:0 novote:12
20 no yes:29 no:155 abstain:2 novote:13
21 yes yes:138 no:43 abstain:0 novote:18
22 yes yes:103 no:71 abstain:0 novote:25
23 yes yes:117 no:57 abstain:0 novote:25
24 yes yes:116 no:53 abstain:0 novote:

In [ ]:
#counts the members of each faction per session
factions = dict()
for mem in members:
    f = members[mem].faction
    for s in members[mem].ismem:
        if s in factions:
            if f in factions[s]:
                factions[s][f] += 1
            else:
                factions[s][f] = 1
        else:
            factions[s] = dict()
            factions[s][f] = 1
  


In [ ]:
for s in factions:
    memsum = 0
    for f in factions[s]:
        if f != 'sum':
            memsum += factions[s][f]
    factions[s]['sum'] = memsum

In [ ]:
def eqDictSet(d):
    out = set()
    

In [ ]:
factions

In [ ]:
def extendSet(s, x):
    for i in x:
        if i != '' and i != 'CouncillorName':
            s.add(i)

In [ ]:
def extendDict(d, l):
    for i in l:
        if i == '' or i == 'Fraktion':
            continue
        if i in d:
            d[i] +=1
        else:
            d[i] = 1

In [ ]:
def parseFile(file):
    lineno = 0
    for line in file:
        votes.add(line[20])
        if lineno == 2:
            extendSet(members, line)
        if lineno == 4:
            extendDict(faction, line)
        if lineno == 8:
            print(line)
        lineno += 1

In [ ]:
len(set())

In [ ]:
print(members['3008'].ismem)

In [ ]:
mh.dictPrint(members)